In [6]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
import ast

# Step 0: Initial Reading of Data
rti_final_df = pd.read_csv('RTI-final.csv')
sa_zones_df = pd.read_csv('SA-zones.csv')

# Step 1: Merging DataFrames
df = pd.merge(rti_final_df, sa_zones_df, how='left', left_on='SA2_CODE21', right_on='0_ZID')


import pandas as pd

# Assuming df['Start_Time'] contains Excel serial date numbers
# Convert Excel serial date numbers to datetime
# df['Start_Time'] = pd.to_datetime(df['Start_Time'], origin='1899-12-30', unit='D')
df['Start_Time'] = pd.to_datetime(df['Start_Time'], format='%d-%m-%Y %H:%M')
# Now you can extract month and hour



# Step 2: Replacing and Evaluating Lists More Safely
# Replace unknown and "[' ']" with "[]"
df['Attending_Groups'] = df['Attending_Groups'].replace({'unknown': '[]', "[' ']": "['Other']"})
df['Attending_Groups'] = df['Attending_Groups'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Step 3: Dropping Unnecessary Columns Early (ensure they're not needed beyond this point)
df = df.drop(['0_ZID', 'LOCI_URI21', 'SA2_NAME21', 'Other_Advice', 'Diversions', 'Attending_Groups', 'End_Time', 'Last_Updated', 'Incident ID'], axis=1)

# Step 4: Extracting Info from Display Name
from sklearn.preprocessing import LabelEncoder
import re

def extract_info_from_display_name(row):
    incident_type = 'unknown'
    num_vehicles_involved = 1  # Default to 1 if specific number is not mentioned

    # Identifying the incident type
    if 'breakdown' in row.lower():
        incident_type = 'breakdown'
    elif 'crash' in row.lower() or 'collision' in row.lower():  # Added 'collision' as a keyword
        incident_type = 'crash'
    elif 'accident' in row.lower():
        incident_type = 'accident'

    # Extract number of cars/vehicles if specified
    numbers = re.findall(r'(\d+)\s*(cars|vehicles)', row, re.IGNORECASE)
    if numbers:
        num_vehicles_involved = int(numbers[0][0])
    elif 'car' in row.lower() or 'vehicle' in row.lower():
        num_vehicles_involved = 1

    # Here you could encode the incident_type using LabelEncoder if needed
    # encoder = LabelEncoder()
    # incident_type_encoded = encoder.fit_transform([incident_type])[0]
    # But for simplicity, we'll return the string type and number as is

    return pd.Series([incident_type, num_vehicles_involved])

# Apply and Create New Columns
df[['Incident_Type', 'Num_Vehicles_Involved']] = df['Display_Name'].apply(extract_info_from_display_name)

# Step 5: Label Encoding
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column].astype(str))
    label_encoders[column] = le

# Extract Month and Hour Information
from datetime import datetime
import pandas as pd

def extract_month_and_hour(start_time_str):
    # Ensure start_time_str is a string
    start_time_str = str(start_time_str)
    
    # Parse the datetime from the string
    dt = datetime.strptime(start_time_str, '%d-%m-%Y %H:%M')
    
    # Return the month and hour as a pandas Series
    return pd.Series([dt.month, dt.hour])

df['Month'] = df['Start_Time'].dt.month
df['Hour'] = df['Start_Time'].dt.hour

# Calculate Distance to CBD
reference_point = (-33.87324469333478, 151.20665137317067)
df['distance_to_CBD'] = df.apply(lambda row: geodesic((row['Latitude'], row['Longitude']), reference_point).km, axis=1)

# Step 6: Identifying Imbalanced Columns & Dropping Them
threshold = 0.95  # Correcting the comment to match the code
imbalanced_columns = [col for col in df.columns if (df[col].value_counts(normalize=True).iloc[0] > threshold)]

df = df.drop(columns=imbalanced_columns)

# Rename and Sanitize Column Names
df.rename(columns={'Duration_in_Minutes': 'duration'}, inplace=True)
df.columns = [col.replace('{', '').replace('}', '').replace('[', '').replace(']', '').replace('"', '').replace(':', '') for col in df.columns]

# Convert Start_Time for Filtering Purposes
# df['Start_Time'] = pd.to_datetime(df['Start_Time'], format='%d-%m-%Y %H:%M')

# Step 7: Filtering Based on Dates
def filter_data_based_on_date(df, start_date=None, end_date=None, before_date=None):
    if start_date and end_date:
        mask = (df['Start_Time'] >= start_date) & (df['Start_Time'] <= end_date)
        return df.loc[mask]
    elif before_date:
        mask = df['Start_Time'] < before_date
        return df.loc[mask]
    return df  # Return original DataFrame if no conditions are met

# Applying Filters
subset_df = filter_data_based_on_date(df, start_date=pd.to_datetime('2020-03-10'), end_date=pd.to_datetime('2022-10-14'))
subset_df_before = filter_data_based_on_date(df, before_date=pd.to_datetime('2020-03-10'))

# Drop the 'Start_Time' column where necessary and exporting to CSV
subset_df.drop(['Start_Time'], axis=1, inplace=True)
subset_df_before.drop(['Start_Time'], axis=1, inplace=True)
subset_df.to_csv('merged_covid.csv', index=False)
subset_df_before.to_csv('merged_before.csv', index=False)

# General Cleanup before Final CSV Export
df.drop(['Start_Time'], axis=1, inplace=True)
df.to_csv('merged.csv', index=False)

C:\Users\A\AppData\Local\Temp\ipykernel_48268\901055969.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df.drop(['Start_Time'], axis=1, inplace=True)
C:\Users\A\AppData\Local\Temp\ipykernel_48268\901055969.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df_before.drop(['Start_Time'], axis=1, inplace=True)
